In [10]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2022_02.parquet')
ts_data= ts_data.reset_index(drop=True)

ts_data.head(20)

,pickup_hour,pickup_location_id,ride_count
0,2022-02-01 00:00:00,43,3
1,2022-02-01 01:00:00,43,1
2,2022-02-01 02:00:00,43,1
3,2022-02-01 03:00:00,43,3
4,2022-02-01 05:00:00,43,6
5,2022-02-01 06:00:00,43,11
6,2022-02-01 07:00:00,43,30
7,2022-02-01 08:00:00,43,76
8,2022-02-01 09:00:00,43,94
9,2022-02-01 10:00:00,43,78


In [3]:
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
) -> list:

    stop_position = len(data) - 1

    # Start the first sub-sequence at index position 0
    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + 1
    indices = []

    while subseq_last_idx <= stop_position:
        indices.append([subseq_first_idx, subseq_mid_idx, subseq_last_idx])

        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size

    return indices


In [5]:
n_features=12
step_size=1

indices=get_cutoff_indices(
    ts_data,
    n_features,
    step_size)

indices[:5]

[[0, 12, 13], [1, 13, 14], [2, 14, 15], [3, 15, 16], [4, 16, 17]]

In [8]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] =  ts_data.iloc[idx[0]:idx[1]]['ride_count'].values
    y[i] = ts_data.iloc[idx[1]:idx[2]]['ride_count'].values[0]

    pickup_hours.append( ts_data.iloc[idx[1]]['pickup_hour'])


In [9]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')


x.shape=(605, 12)
x=array([[  3.,   1.,   1., ...,  78., 115., 119.],
       [  1.,   1.,   3., ..., 115., 119., 117.],
       [  1.,   3.,   6., ..., 119., 117., 153.],
       ...,
       [ 30.,  68., 105., ..., 153., 112.,  37.],
       [ 68., 105., 140., ..., 112.,  37.,  30.],
       [105., 140., 132., ...,  37.,  30.,  34.]],
      shape=(605, 12), dtype=float32)
pickup_hours[:5]=[Timestamp('2022-02-01 13:00:00'), Timestamp('2022-02-01 14:00:00'), Timestamp('2022-02-01 15:00:00'), Timestamp('2022-02-01 16:00:00'), Timestamp('2022-02-01 17:00:00')]


In [ ]:
features_location_CP = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)

features_location_CP



,rides_previous_12_hour,rides_previous_11_hour,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,3.0,1.0,1.0,3.0,6.0,11.0,30.0,76.0,94.0,78.0,115.0,119.0
1,1.0,1.0,3.0,6.0,11.0,30.0,76.0,94.0,78.0,115.0,119.0,117.0
2,1.0,3.0,6.0,11.0,30.0,76.0,94.0,78.0,115.0,119.0,117.0,153.0
3,3.0,6.0,11.0,30.0,76.0,94.0,78.0,115.0,119.0,117.0,153.0,136.0
4,6.0,11.0,30.0,76.0,94.0,78.0,115.0,119.0,117.0,153.0,136.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...
600,10.0,17.0,30.0,68.0,105.0,140.0,132.0,188.0,162.0,199.0,223.0,153.0
601,17.0,30.0,68.0,105.0,140.0,132.0,188.0,162.0,199.0,223.0,153.0,112.0
602,30.0,68.0,105.0,140.0,132.0,188.0,162.0,199.0,223.0,153.0,112.0,37.0
603,68.0,105.0,140.0,132.0,188.0,162.0,199.0,223.0,153.0,112.0,37.0,30.0


In [12]:
targets_location_CP = pd.DataFrame(
    y,
    columns=[f'target_rides_next_hour']
)

targets_location_CP

,target_rides_next_hour
0,117.0
1,153.0
2,136.0
3,99.0
4,119.0
...,...
600,112.0
601,37.0
602,30.0
603,34.0
